# LSTM+CRF序列标注

## 概述

序列标注指给定输入序列，给序列中每个Token进行标注标签的过程。序列标注问题通常用于从文本中进行信息抽取，包括分词(Word Segmentation)、词性标注(Position Tagging)、命名实体识别(Named Entity Recognition, NER)等。以命名实体识别为例：

| 输入序列 | 清 | 华 | 大 | 学 | 座 | 落 | 于 | 首 | 都 | 北 | 京 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
|输出标注| B | I | I | I | O | O | O | O | O | B | I |

如上表所示，`清华大学` 和 `北京`是地名，需要将其识别，我们对每个输入的单词预测其标签，最后根据标签来识别实体。

> 这里使用了一种常见的命名实体识别的标注方法——“BIOE”标注，将一个实体(Entity)的开头标注为B，其他部分标注为I，非实体标注为O。

## 条件随机场(Conditional Random Field, CRF)

从上文的举例可以看到，对序列进行标注，实际上是对序列中每个Token进行标签预测，可以直接视作简单的多分类问题。但是序列标注不仅仅需要对单个Token进行分类预测，同时相邻Token直接有关联关系。以`清华大学`一词为例:

| 输入序列 | 清 | 华 | 大 | 学 | |
| --- | --- | --- | --- | --- | --- |
| 输出标注 | B | I | I | I | √ |
| 输出标注 | O | I | I | I | × |

如上表所示，正确的实体中包含的4个Token有依赖关系，I前必须是B或I，而错误输出结果将`清`字标注为O，违背了这一依赖。将命名实体识别视为多分类问题，则每个词的预测概率都是独立的，易产生类似的问题，因此需要引入一种能够学习到此种关联关系的算法来保证预测结果的正确性。而条件随机场是适合此类场景的一种[概率图模型](https://en.wikipedia.org/wiki/Graphical_model)。下面对条件随机场的定义和参数化形式进行简析。

> 考虑到序列标注问题的线性序列特点，本节所述的条件随机场特指线性链条件随机场(Linear Chain CRF)

设$x=\{x_0, ..., x_n\}$为输入序列，$y=\{y_0, ..., y_n\}, y \in Y$为输出的标注序列，其中$n$为序列的最大长度，$Y$表示$x$对应的所有可能的输出序列集合。则输出序列$y$的概率为：

$$\begin{align}P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y' \in Y} \exp{(\text{Score}(x, y')})} \qquad (1)\end{align}$$

设$x_i$, $y_i$为序列的第$i$个Token和对应的标签，则$\text{Score}$需要能够在计算$x_i$和$y_i$的映射的同时，捕获相邻标签$y_{i-1}$和$y_{i}$之间的关系，因此我们定义两个概率函数：

1. 发射概率函数$\psi_\text{EMIT}$：表示$x_i \rightarrow y_i$的概率。
2. 转移概率函数$\psi_\text{TRANS}$：表示$y_{i-1} \rightarrow y_i$的概率。

则可以得到$\text{Score}$的计算公式：

$$\begin{align}\text{Score}(x,y) = \sum_i \log \psi_\text{EMIT}(x_i \rightarrow y_i) + \log \psi_\text{TRANS}(y_{i-1} \rightarrow y_i) \qquad (2)\end{align} $$

设标签集合为$T$, 构造大小为$|T|x|T|$的矩阵$\textbf{P}$,用于存储标签间的转移概率；由编码层(可以为Dense、LSTM等)输出的隐状态$h$可以直接视作发射概率，此时$\text{Score}$的计算公式可以转化为：

$$\begin{align}\text{Score}(x,y) = \sum_i h_i[y_i] + \textbf{P}_{y_{i-1}, y_{i}} \qquad (3)\end{align}$$



> 完整的CRF完整推导可参考[Log-Linear Models, MEMMs, and CRFs](http://www.cs.columbia.edu/~mcollins/crf.pdf)


接下来我们根据上述公式，使用MindSpore来实现CRF的参数化形式。首先实现CRF层的前向训练部分，将CRF和损失函数做合并，选择分类问题常用的负对数似然函数(Negative Log Likelihood, NLL)，则有：

$$\begin{align}\text{Loss} = -log(P(y|x)) \qquad (4)\end{align} $$

由公式$(1)$可得，

$$\begin{align}\text{Loss} = -log(\frac{\exp{(\text{Score}(x, y)})}{\sum_{y' \in Y} \exp{(\text{Score}(x, y')})}) \qquad (5)\end{align} $$

$$\begin{align}= log(\sum_{y' \in Y} \exp{(\text{Score}(x, y')}) - \text{Score}(x, y) \end{align}$$

根据公式$(5)$，我们称被减数为Normalizer，减数为Score，分别实现后相减得到最终Loss。

### Score计算

首先根据公式$(3)$计算正确标签序列所对应的得分，这里需要注意，除了转移概率矩阵$\textbf{P}$外，还需要维护两个大小为$|T|$的向量，分别作为序列开始和结束时的转移概率。同时我们引入了一个掩码矩阵$mask$，将多个序列打包为一个Batch时填充的值忽略，使得$\text{Score}$计算仅包含有效的Token。

In [22]:
def compute_score(emissions, tags, seq_ends, mask, trans, start_trans, end_trans):
    # emissions: (seq_length, batch_size, num_tags)
    # tags: (seq_length, batch_size)
    # mask: (seq_length, batch_size)

    seq_length, batch_size = tags.shape
    mask = mask.astype(emissions.dtype)

    # 将score设置为初始转移概率
    # shape: (batch_size,)
    score = start_trans[tags[0]]
    # score += 第一次发射概率
    # shape: (batch_size,)
    score += emissions[0, mnp.arange(batch_size), tags[0]]

    for i in range(1, seq_length):
        # 标签由i-1转移至i的转移概率（当mask == 1时有效）
        # shape: (batch_size,)
        score += trans[tags[i - 1], tags[i]] * mask[i]

        # 预测tags[i]的发射概率（当mask == 1时有效）
        # shape: (batch_size,)
        score += emissions[i, mnp.arange(batch_size), tags[i]] * mask[i]

    # 结束转移
    # shape: (batch_size,)
    last_tags = tags[seq_ends, mnp.arange(batch_size)]
    # score += 结束转移概率
    # shape: (batch_size,)
    score += end_trans[last_tags]

    return score

### Normalizer计算

根据公式$(5)$，Normalizer是$x$对应的所有可能的输出序列的Score的对数指数和(Log-Sum-Exp)。此时如果按穷举法进行计算，则需要将每个可能的输出序列Score都计算一遍，共有$|T|^{n}$个结果。这里我们采用动态规划算法，通过复用计算结果来提高效率。

假设需要计算从第$0$至第$i$个Token所有可能的输出序列得分$\text{Score}_{i}$，则可以先计算出从第$0$至第$i-1$个Token所有可能的输出序列得分$\text{Score}_{i-1}$。因此，Normalizer可以改写为以下形式：

$$log(\sum_{y'_{0,i} \in Y} \exp{(\text{Score}_i})) = log(\sum_{y'_{0,i-1} \in Y} \exp{(\text{Score}_{i-1} + h_{i} + \textbf{P}})) \qquad (6)$$

其中$h_i$为第$i$个Token的发射概率，$\textbf{P}$是转移矩阵。由于发射概率矩阵$h$和转移概率矩阵$\textbf{P}$独立于$y$的序列路径计算，可以将其提出，可得：

$$log(\sum_{y'_{0,i} \in Y} \exp{(\text{Score}_i})) = log(\sum_{y'_{0,i-1} \in Y} \exp{(\text{Score}_{i-1}})) + h_{i} + \textbf{P} \qquad (7)$$

根据公式(7)，Normalizer的实现如下：

In [23]:
def compute_normalizer(emissions, mask, trans, start_trans, end_trans):
    # emissions: (seq_length, batch_size, num_tags)
    # mask: (seq_length, batch_size)

    seq_length = emissions.shape[0]

    # 将score设置为初始转移概率，并加上第一次发射概率
    # shape: (batch_size, num_tags)
    score = start_trans + emissions[0]

    for i in range(1, seq_length):
        # 扩展score的维度用于总score的计算
        # shape: (batch_size, num_tags, 1)
        broadcast_score = score.expand_dims(2)

        # 扩展emission的维度用于总score的计算
        # shape: (batch_size, 1, num_tags)
        broadcast_emissions = emissions[i].expand_dims(1)

        # 根据公式(7)，计算score_i
        # 此时broadcast_score是由第0个到当前Token所有可能路径
        # 对应score的log_sum_exp
        # shape: (batch_size, num_tags, num_tags)
        next_score = broadcast_score + trans + broadcast_emissions

        # 对score_i做log_sum_exp运算，用于下一个Token的score计算
        # shape: (batch_size, num_tags)
        next_score = ops.logsumexp(next_score, axis=1)

        # 当mask == 1时，score才会变化
        # shape: (batch_size, num_tags)
        score = mnp.where(mask[i].expand_dims(1), next_score, score)

    # 最后加结束转移概率
    # shape: (batch_size, num_tags)
    score += end_trans
    # 对所有可能的路径得分求log_sum_exp
    # shape: (batch_size,)
    return ops.logsumexp(score, axis=1)

### Viterbi算法

在完成前向训练部分后，需要实现解码部分。这里我们选择适合求解序列最优路径的[Viterbi算法](https://en.wikipedia.org/wiki/Viterbi_algorithm)。与计算Normalizer类似，使用动态规划求解所有可能的预测序列得分。不同的是在解码时同时需要将第$i$个Token对应的score取值最大的标签保存，供后续使用Viterbi算法求解最优预测序列使用。

取得最大概率得分$\text{Score}$，以及每个Token对应的标签历史$\text{History}$后，根据Viterbi算法可以得到公式：

$$P_{0,i} = max(P_{0, i-1}) + P_{i-1, i}$$

从第0个至第$i$个Token对应概率最大的序列，只需要考虑从第0个至第$i-1$个Token对应概率最大的序列，以及从第$i$个至第$i-1$个概率最大的标签即可。因此我们逆序求解每一个概率最大的标签，构成最佳的预测序列。

> 由于静态图语法限制，我们将Viterbi算法求解最佳预测序列的部分作为后处理函数，不纳入后续CRF层的实现。

In [24]:
def viterbi_decode(emissions, mask, trans, start_trans, end_trans):
    # emissions: (seq_length, batch_size, num_tags)
    # mask: (seq_length, batch_size)

    seq_length = mask.shape[0]

    score = start_trans + emissions[0]
    history = ()

    for i in range(1, seq_length):
        broadcast_score = score.expand_dims(2)
        broadcast_emission = emissions[i].expand_dims(1)
        next_score = broadcast_score + trans + broadcast_emission

        # 求当前Token对应score取值最大的标签，并保存
        indices = next_score.argmax(axis=1)
        history += (indices,)

        next_score = next_score.max(axis=1)
        score = mnp.where(mask[i].expand_dims(1), next_score, score)

    score += end_trans

    return score, history

def post_decode(score, history, seq_length):
    # 使用Score和History计算最佳预测序列
    batch_size = seq_length.shape[0]
    seq_ends = seq_length - 1
    # shape: (batch_size,)
    best_tags_list = []

    # 依次对一个Batch中每个样例进行解码
    for idx in range(batch_size):
        # 查找使最后一个Token对应的预测概率最大的标签，
        # 并将其添加至最佳预测序列存储的列表中
        best_last_tag = score[idx].argmax(axis=0)
        best_tags = [int(best_last_tag.asnumpy())]

        # 重复查找每个Token对应的预测概率最大的标签，加入列表
        for hist in reversed(history[:seq_ends[idx]]):
            best_last_tag = hist[idx][best_tags[-1]]
            best_tags.append(int(best_last_tag.asnumpy()))

        # 将逆序求解的序列标签重置为正序
        best_tags.reverse()
        best_tags_list.append(best_tags)

    return best_tags_list

### CRF层

完成上述前向训练和解码部分的代码后，将其组装完整的CRF层。考虑到输入序列可能存在Padding的情况，CRF的输入需要考虑输入序列的真实长度，因此除发射矩阵和标签外，加入`seq_length`参数传入序列Padding前的长度，并实现生成mask矩阵的`sequence_mask`方法。

综合上述代码，使用`nn.Cell`进行封装，最后实现完整的CRF层如下：

In [25]:
import mindspore as ms
import mindspore.nn as nn
import mindspore.ops as ops
import mindspore.numpy as mnp
from mindspore.common.initializer import initializer, Uniform

def sequence_mask(seq_length, max_length, batch_first=False):
    """根据序列实际长度和最大长度生成mask矩阵"""
    range_vector = mnp.arange(0, max_length, 1, seq_length.dtype)
    result = range_vector < seq_length.view(seq_length.shape + (1,))
    if batch_first:
        return result.astype(ms.int64)
    return result.astype(ms.int64).swapaxes(0, 1)

class CRF(nn.Cell):
    def __init__(self, num_tags: int, batch_first: bool = False, reduction: str = 'sum') -> None:
        if num_tags <= 0:
            raise ValueError(f'invalid number of tags: {num_tags}')
        super().__init__()
        if reduction not in ('none', 'sum', 'mean', 'token_mean'):
            raise ValueError(f'invalid reduction: {reduction}')
        self.num_tags = num_tags
        self.batch_first = batch_first
        self.reduction = reduction
        self.start_transitions = ms.Parameter(initializer(Uniform(0.1), (num_tags,)), name='start_transitions')
        self.end_transitions = ms.Parameter(initializer(Uniform(0.1), (num_tags,)), name='end_transitions')
        self.transitions = ms.Parameter(initializer(Uniform(0.1), (num_tags, num_tags)), name='transitions')

    def construct(self, emissions, tags=None, seq_length=None):
        if tags is None:
            return self._decode(emissions, seq_length)
        return self._forward(emissions, tags, seq_length)

    def _forward(self, emissions, tags=None, seq_length=None):
        if self.batch_first:
            batch_size, max_length = tags.shape
            emissions = emissions.swapaxes(0, 1)
            tags = tags.swapaxes(0, 1)
        else:
            max_length, batch_size = tags.shape

        if seq_length is None:
            seq_length = mnp.full((batch_size,), max_length, ms.int64)

        mask = sequence_mask(seq_length, max_length)

        # shape: (batch_size,)
        numerator = compute_score(emissions, tags, seq_length-1, mask, self.transitions, self.start_transitions, self.end_transitions)
        # shape: (batch_size,)
        denominator = compute_normalizer(emissions, mask, self.transitions, self.start_transitions, self.end_transitions)
        # shape: (batch_size,)
        llh = denominator - numerator

        if self.reduction == 'none':
            return llh
        if self.reduction == 'sum':
            return llh.sum()
        if self.reduction == 'mean':
            return llh.mean()
        return llh.sum() / mask.astype(emissions.dtype).sum()

    def _decode(self, emissions, seq_length=None):
        if self.batch_first:
            batch_size, max_length = emissions.shape[:2]
            emissions = emissions.swapaxes(0, 1)
        else:
            batch_size, max_length = emissions.shape[:2]

        if seq_length is None:
            seq_length = mnp.full((batch_size,), max_length, ms.int64)

        mask = sequence_mask(seq_length, max_length)

        return viterbi_decode(emissions, mask, self.transitions, self.start_transitions, self.end_transitions)

## BiLSTM+CRF模型

在实现CRF后，我们设计一个双向LSTM+CRF的模型来进行命名实体识别任务的训练。模型结构如下：

```text
nn.Embedding -> nn.LSTM -> nn.Dense -> CRF
```

其中LSTM提取序列特征，经过Dense层变换获得发射概率矩阵，最后送入CRF层。具体实现如下：

In [26]:
class BiLSTM_CRF(nn.Cell):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_tags, padding_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Dense(hidden_dim, num_tags, 'he_uniform')
        self.crf = CRF(num_tags, batch_first=True)

    def construct(self, inputs, seq_length, tags=None):
        embeds = self.embedding(inputs)
        outputs, _ = self.lstm(embeds, seq_length=seq_length)
        feats = self.hidden2tag(outputs)

        crf_outs = self.crf(feats, tags, seq_length)
        return crf_outs

完成模型设计后，我们生成两句例子和对应的标签，并构造词表和标签表。

In [27]:
embedding_dim = 16
hidden_dim = 32

training_data = [(
    "我 爱 长 沙 的 臭 豆 腐".split(),
    "O O B I O O O O".split()
), (
    "      ".split(),
    "      ".split()
)]

word_to_idx = {}
word_to_idx['<pad>'] = 0
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)

tag_to_idx = {"B": 0, "I": 1, "O": 2}

In [2]:
len(word_to_idx)

NameError: name 'word_to_idx' is not defined

接下来实例化模型，选择优化器并将模型和优化器送入Wrapper。

> 由于CRF层已经进行了NLLLoss的计算，因此不需要再设置Loss。

In [29]:
model = BiLSTM_CRF(len(word_to_idx), embedding_dim, hidden_dim, len(tag_to_idx))
optimizer = nn.SGD(model.trainable_params(), learning_rate=0.01, weight_decay=1e-4)

In [1]:
grad_fn = ops.value_and_grad(model, None, optimizer.parameters)

def train_step(data, seq_length, label):
    loss, grads = grad_fn(data, seq_length, label)
    optimizer(grads)
    return loss

NameError: name 'ms' is not defined

将生成的数据打包成Batch，按照序列最大长度，对长度不足的序列进行填充，分别返回输入序列、输出标签和序列长度构成的Tensor。

In [31]:
def prepare_sequence(seqs, word_to_idx, tag_to_idx):
    seq_outputs, label_outputs, seq_length = [], [], []
    max_len = max([len(i[0]) for i in seqs])

    for seq, tag in seqs:
        seq_length.append(len(seq))
        idxs = [word_to_idx[w] for w in seq]
        labels = [tag_to_idx[t] for t in tag]
        idxs.extend([word_to_idx['<pad>'] for i in range(max_len - len(seq))])
        labels.extend([tag_to_idx['O'] for i in range(max_len - len(seq))])
        seq_outputs.append(idxs)
        label_outputs.append(labels)

    return ms.Tensor(seq_outputs, ms.int64), \
            ms.Tensor(label_outputs, ms.int64), \
            ms.Tensor(seq_length, ms.int64)

In [32]:
data, label, seq_length = prepare_sequence(training_data, word_to_idx, tag_to_idx)
data.shape, label.shape, seq_length.shape

((2, 12), (2, 12), (2,))

对模型进行预编译后，训练500个step。

> 训练流程可视化依赖`tqdm`库，可使用```pip install tqdm```命令安装。

In [33]:
from tqdm import tqdm

steps = 500
with tqdm(total=steps) as t:
    for i in range(steps):
        loss = train_step(data, seq_length, label)
        t.set_postfix(loss=loss)
        t.update(1)

100%|███████████████████████████████████████████████████████████████| 500/500 [00:29<00:00, 17.22it/s, loss=0.30596733]


最后我们来观察训练500个step后的模型效果，首先使用模型预测可能的路径得分以及候选序列。

In [34]:
score, history = model(data, seq_length)
score

[WARNING] ME(19248:4540,MainProcess):2023-05-27-20:26:03.425.208 [mindspore\ops\primitive.py:713] The "_check_is_tensor" is a constexpr function. The input arguments must be all constant value.


Tensor(shape=[2, 3], dtype=Float32, value=
[[2.34930553e+001, 2.02359905e+001, 2.93754025e+001],
 [4.41840897e+001, 4.09088097e+001, 5.03313942e+001]])

使用后处理函数进行预测得分的后处理。

In [35]:
predict = post_decode(score, history, seq_length)
predict

[[2, 2, 0, 1, 2, 2, 2, 2], [0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]]

最后将预测的index序列转换为标签序列，打印输出结果，查看效果。

In [36]:
idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}

def sequence_to_tag(sequences, idx_to_tag):
    outputs = []
    for seq in sequences:
        outputs.append([idx_to_tag[i] for i in seq])
    return outputs

In [37]:
sequence_to_tag(predict, idx_to_tag)

[['O', 'O', 'B', 'I', 'O', 'O', 'O', 'O'],
 ['B', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]